In [1]:
import time
import csv
import pickle
import pandas as pd
import numpy as np
import operator
import os

In [2]:
#Only select session which has final purchase
purchase = pd.read_csv('../diginetica/train-purchases.csv', sep=';')[['sessionId', 'itemId']]
print('Total purchase', len(purchase))
queries = pd.read_csv('../diginetica/train-queries.csv', sep=';')[['queryId', 'sessionId', 'items']]
print('Total queries', len(queries))
clicks = pd.read_csv('../diginetica/train-clicks.csv', sep=';')[['queryId', 'itemId']]
print('Total clicks', len(clicks))

Total purchase 18025


/home/xb6cf/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Total queries 923127
Total clicks 1127764


In [3]:
#Query-Session Dictionary
class QS_Dictionary(object):
    def __init__(self):
        self.query_dic={}
        
    def add_session(self, query, session_id):
        self.query_dic[query]=session_id
            
    def get_Dic(self):
        return self.query_dic
    
    def del_key(self,session_id):
        for key in self.query_dic: 
            if self.query_dic[key] == session_id:
                self.query_dic.pop(key)
    
    def length(self):
        return len(self.query_dic)

In [4]:
#Session-click dictionary
class SC_Dictionary(object):
    def __init__(self):
        self.session_dic={}
        
    def add_click(self, query, click, qs_dictionary):
        if query in qs_dictionary.get_Dic():
            session_id = qs_dictionary.get_Dic()[query]
            if session_id not in self.session_dic:
                self.session_dic[session_id] = [click]
            else:
                self.session_dic[session_id].append(click)
                
    def get_Dic(self):
        return self.session_dic
    
    def del_key(self,key):
        self.session_dic.pop(key)
    
    def length(self):
        return len(self.session_dic)

In [5]:
#Session-Purchase Dictionary
class SP_Dictionary(object):
    def __init__(self):
        self.session_dic={}
        self.multiple=False
        self.unique = 0
        
    def add_item(self, session_id, item_purchase):
        if session_id not in self.session_dic:
            self.session_dic[session_id]=[item_purchase]
            self.unique+=1
        else:
            self.session_dic[session_id].append(item_purchase)
            self.multiple=True
        
    def add_item_unique(self, session_id, item_purchase):
        if session_id not in self.session_dic:
            self.session_dic[session_id]=[item_purchase]
            self.unique+=1
        else: #Remove keys with multiple purchases
            self.session_dic.pop(session_id)
            self.unique=self.unique-1
            
    def If_multiple(self):
        if self.multiple:
            print("Number of sessions with only one purchase is:")
            print(self.unique)
        return self.multiple
    
    def get_Dic(self):
        return self.session_dic
    
    def del_key(self,key):
        self.session_dic.pop(key)
    
    def length(self):
        return len(self.session_dic)

In [6]:
query_session = QS_Dictionary()
session_click = SC_Dictionary()
session_purchase = SP_Dictionary()

for sp in purchase.values:
    session_purchase.add_item(sp[0], sp[1])
    #session_purchase.add_item_unique(sp[0], sp[1])
for qsr in queries.values:
    query_session.add_session(qsr[0], qsr[1])
for qc in clicks.values:
    session_click.add_click(qc[0], qc[1], query_session)
    
#Filter sessions by clicks
filter_key_click = []
#The session has less than 2 clicks        
for key in session_click.get_Dic():
    if len(session_click.get_Dic()[key]) < 2:
        filter_key_click.append(key)
for key in filter_key_click:
    session_click.del_key(key)

In [7]:
class Dictionary(object):
    def __init__(self):
        self.item_dic={}
        #Item number start from 1; if 0, them it means no item selected
        self.current = 1
        
    def add_item(self, item_id):
        if item_id not in self.item_dic:
            self.item_dic[item_id]=self.current
            self.current += 1
            
    def getItem(self):
        return self.item_dic    
    
    def length(self):
        return len(self.item_dic)

In [8]:
##Extract session recommend and clicks; relation between views and clicks?
class Session(object):
    """Instance that represent a sample of train/dev/test corpus."""

    def __init__(self, session_id):
        self.session_id = session_id
        self.clicks = []  
        self.purchase = []
                
    def Add_click(self, session_id, session_click, item_dic):
        assert session_id == self.session_id
        clicks=session_click[session_id]
        #Update the dictionary
        for item in clicks:
            item_dic.add_item(item)
            self.clicks.append(item_dic.getItem()[item])
        
    def Add_purchase(self, session_id, session_purchase, item_dic):
        assert session_id == self.session_id
        if session_id in session_purchase:
            purchases=session_purchase[session_id]
            #Update the dictionary
            for item in purchases:
                item_dic.add_item(item)
                self.purchase.append(item_dic.getItem()[item])
        else:
            #No item purchased, denoted as -1
            self.purchase=[-1]
        
    def write_session_clicks(self, file):
        for cli in range(len(self.clicks)):
            file.write(str(self.clicks[cli]) + ' ')
        file.write('0\n')
        
    def write_session_purchases(self, file):
        for pur in range(len(self.purchase)):
            file.write(str(self.purchase[pur]) + ' ')
        file.write('\n')

In [9]:
def Sample(session_click, session_purchase, item_dic):
    session_list = []
    saveclick=open(os.path.join('../','dig_seq_click_rl.txt'),'a+')
    savepurchase=open(os.path.join('../','dig_seq_purchase_rl.txt'),'a+')
    for session_id in session_click.get_Dic():
        sess = Session(session_id)
        sess.Add_click(session_id, session_click.get_Dic(), item_dic)
        sess.Add_purchase(session_id, session_purchase.get_Dic(), item_dic)
        sess.write_session_clicks(saveclick)
        sess.write_session_purchases(savepurchase)
        session_list.append(sess)
    return session_list

In [10]:
item_dic=Dictionary()
session_list = Sample(session_click, session_purchase, item_dic)

In [17]:
len(session_list)

194771

In [24]:
session_purchase.length()

12630

In [27]:
item_dic.length()

72237